# Simple ring parameter constraints

This notebook provides test cases for the precision of a simple lens model (with simplified assumptions). This is for show-casing and to assess the uncertainty limit in how well the parameters of this model can be constrained.


In [ ]:
# import of standard python libraries
import copy
import numpy as np
import matplotlib.pyplot as plt

# import main simulation class of lenstronomy
from lenstronomy.Util import util, image_util
from lenstronomy.Data.psf import PSF
from lenstronomy.Workflow.fitting_sequence import FittingSequence as FittingSequence_ref

from jaxtronomy.Workflow.fitting_sequence import FittingSequence
from jaxtronomy.LensModel.lens_model import LensModel
from jaxtronomy.LightModel.light_model import LightModel
from jaxtronomy.PointSource.point_source import PointSource
from jaxtronomy.Data.imaging_data import ImageData
from jaxtronomy.ImSim.image_model import ImageModel

# Currently, jaxtronomy supports the following deflector and source profiles:
from jaxtronomy.LensModel.profile_list_base import (
    _JAXXED_MODELS as _JAXXED_DEFLECTOR_MODELS,
)
from jaxtronomy.LightModel.light_model_base import (
    _JAXXED_MODELS as _JAXXED_LIGHT_MODELS,
)

print("Deflector models:", _JAXXED_DEFLECTOR_MODELS)
print("Light models:", _JAXXED_LIGHT_MODELS)

## simulation choices

In [ ]:
# data specifics
background_rms = 0.015  #  background noise per pixel
exp_time = 500.0  #  exposure time (arbitrary units, flux per pixel is in units #photons/exp_time unit)
numPix = 100  #  cutout pixel size per axis
pixel_scale = 0.11  #  pixel size in arcsec (area per pixel = pixel_scale**2)
fwhm = 0.3  # full width at half maximum of PSF

# lensing quantities
lens_model_list = ["EPL", "SHEAR"]
kwargs_epl = {
    "theta_E": 2.66,
    "gamma": 1.7,
    "e1": 0.07,
    "e2": -0.03,
    "center_x": 0.01,
    "center_y": -0.01,
}
kwargs_shear = {"gamma1": 0.0, "gamma2": -0.05}

if lens_model_list[0] == "SIE":
    kwargs_epl.pop("gamma")

kwargs_lens = [kwargs_epl, kwargs_shear]
lens_model_class = LensModel(lens_model_list)


# Sersic parameters in the initial simulation for the source
kwargs_sersic = {
    "amp": 26.0,
    "R_sersic": 1.1,
    "n_sersic": 1.0,
    "e1": -0.1,
    "e2": 0.1,
    "center_x": 0.01,
    "center_y": -0.01,
}
source_model_list = ["SERSIC_ELLIPSE"]

kwargs_source = [kwargs_sersic]
source_model_class = LightModel(source_model_list)


kwargs_sersic_lens = {
    "amp": 16.0,
    "R_sersic": 3.6,
    "n_sersic": 2.0,
    "e1": -0.1,
    "e2": 0.1,
    "center_x": 0.01,
    "center_y": -0.01,
}

lens_light_model_list = ["SERSIC_ELLIPSE"]
kwargs_lens_light = [kwargs_sersic_lens]

lens_light_model_class = LightModel(lens_light_model_list)

point_source_type_list = ["LENSED_POSITION"]
kwargs_lensed_position = {
    "ra_image": [0.69920543, -0.69893762, -2.44616562, 2.42290681],
    "dec_image": [-2.78394548, 2.72912308, -0.41755671, 0.55430711],
    "source_amp": 2,
}
kwargs_ps = [kwargs_lensed_position]
point_source_class = PointSource(
    point_source_type_list, lens_model_class, fixed_magnification_list=[True]
)

kwargs_truth = {
    "kwargs_lens": kwargs_lens,
    "kwargs_source": kwargs_source,
    "kwargs_lens_light": kwargs_lens_light,
    "kwargs_ps": kwargs_ps,
}

## Simulating a mock lens
In the blocks below we simulate a mock lens to generate an image. We only require the kwargs_data and kwargs_psf arguments to perform the modeling. If you have real data, you can leave out the image simulation and directly read in the data, PSF and noise properties into the keyword argument list. Make sure the units are correct. Further information on the settings are available in the ImageData() and PSF() classes in the lenstronomy.Data module.

In [ ]:
# generate the coordinate grid and image properties (we only read out the relevant lines we need)
_, _, ra_at_xy_0, dec_at_xy_0, _, _, Mpix2coord, _ = util.make_grid_with_coordtransform(
    numPix=numPix,
    deltapix=pixel_scale,
    center_ra=0,
    center_dec=0,
    subgrid_res=1,
    inverse=False,
)


kwargs_data = {
    "background_rms": background_rms,  # rms of background noise
    "exposure_time": exp_time,  # exposure time (or a map per pixel)
    "ra_at_xy_0": ra_at_xy_0,  # RA at (0,0) pixel
    "dec_at_xy_0": dec_at_xy_0,  # DEC at (0,0) pixel
    "transform_pix2angle": Mpix2coord,  # matrix to translate shift in pixel in shift in relative RA/DEC (2x2 matrix). Make sure it's units are arcseconds or the angular units you want to model.
    "image_data": np.zeros(
        (numPix, numPix)
    ),  # 2d data vector, here initialized with zeros as place holders that get's overwritten once a simulated image with noise is created.
}
data_class = ImageData(**kwargs_data)


# generate the psf variables
kwargs_psf_gaussian = {"psf_type": "GAUSSIAN", "pixel_size": pixel_scale, "fwhm": fwhm}
psf_class = PSF(**kwargs_psf_gaussian)


kwargs_numerics = {
    "supersampling_factor": 1,
    "supersampling_convolution": False,
    "convolution_type": "fft_static",
}


# create ImageModel class and generate image
imageModel = ImageModel(
    data_class,
    psf_class,
    lens_model_class=lens_model_class,
    source_model_class=source_model_class,
    lens_light_model_class=lens_light_model_class,
    point_source_class=point_source_class,
    kwargs_numerics=kwargs_numerics,
)
image_model = imageModel.image(
    kwargs_lens,
    kwargs_source,
    kwargs_lens_light=kwargs_lens_light,
    kwargs_ps=kwargs_ps,
)

# Can add noise if desired
poisson = image_util.add_poisson(image_model, exp_time=exp_time)
bkg = image_util.add_background(image_model, sigma_bkd=background_rms)
image_model += poisson + bkg

kwargs_data["image_data"] = image_model

data_class.update_data(image_model)

plt.imshow(image_model)

## Defining a Prior Distribution
Now that we have our data image, our goal is to sample through the parameter space to recover the true parameters of the lens that generated this image. First, we define a prior distribution to pass into our samplers.

In [ ]:
# deflector priors
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []

# Since our deflector model consists of ["EPL", "SHEAR"], we first define priors for the EPL
fixed_lens.append({})
kwargs_lens_init.append(
    {
        "theta_E": 2.0,
        "gamma": 2.1,
        "e1": 0.05,
        "e2": -0.05,
        "center_x": 0.1,
        "center_y": 0.2,
    }
)
kwargs_lens_sigma.append(
    {
        "theta_E": 0.1,
        "gamma": 0.1,
        "e1": 0.01,
        "e2": 0.01,
        "center_x": 0.1,
        "center_y": 0.1,
    }
)
kwargs_lower_lens.append(
    {
        "theta_E": 0.01,
        "gamma": 1.5,
        "e1": -0.5,
        "e2": -0.5,
        "center_x": -10.0,
        "center_y": -10.0,
    }
)
kwargs_upper_lens.append(
    {
        "theta_E": 10.0,
        "gamma": 2.5,
        "e1": 0.5,
        "e2": 0.5,
        "center_x": 10.0,
        "center_y": 10.0,
    }
)

if lens_model_list[0] == "SIE":
    kwargs_lens_init[0].pop("gamma")
    kwargs_lens_sigma[0].pop("gamma")
    kwargs_lower_lens[0].pop("gamma")
    kwargs_upper_lens[0].pop("gamma")

# Now we define priors for the SHEAR
fixed_lens.append({"ra_0": 0, "dec_0": 0})
kwargs_lens_init.append({"gamma1": 0.1, "gamma2": 0.04})
kwargs_lens_sigma.append({"gamma1": 0.01, "gamma2": 0.01})
kwargs_lower_lens.append({"gamma1": -0.2, "gamma2": -0.2})
kwargs_upper_lens.append({"gamma1": 0.2, "gamma2": 0.2})

lens_params = [
    kwargs_lens_init,
    kwargs_lens_sigma,
    fixed_lens,
    kwargs_lower_lens,
    kwargs_upper_lens,
]


# source priors
fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []

fixed_source.append({})
kwargs_source_init.append(
    {
        "amp": 16,
        "e1": -0.1,
        "e2": 0.1,
        "R_sersic": 1.2,
        "n_sersic": 1.5,
        "center_x": 0.3,
        "center_y": 0.0,
    }
)
kwargs_source_sigma.append(
    {
        "amp": 1,
        "e1": 0.01,
        "e2": 0.01,
        "R_sersic": 0.01,
        "n_sersic": 0.1,
        "center_x": 0.1,
        "center_y": 0.1,
    }
)
kwargs_lower_source.append(
    {
        "amp": 0,
        "e1": -0.5,
        "e2": -0.5,
        "R_sersic": 0.001,
        "n_sersic": 0.5,
        "center_x": -10.0,
        "center_y": -10.0,
    }
)
kwargs_upper_source.append(
    {
        "amp": 100,
        "e1": 0.5,
        "e2": 0.5,
        "R_sersic": 10,
        "n_sersic": 5.0,
        "center_x": 10,
        "center_y": 10,
    }
)

source_params = [
    kwargs_source_init,
    kwargs_source_sigma,
    fixed_source,
    kwargs_lower_source,
    kwargs_upper_source,
]


# Lens light priors
fixed_lens_light = []
kwargs_lens_light_init = []
kwargs_lens_light_sigma = []
kwargs_lower_lens_light = []
kwargs_upper_lens_light = []

fixed_lens_light.append({})
kwargs_lens_light_init.append(
    {
        "amp": 10,
        "e1": -0.05,
        "e2": 0.05,
        "R_sersic": 2.5,
        "n_sersic": 2.0,
        "center_x": 0.1,
        "center_y": 0.0,
    }
)
kwargs_lens_light_sigma.append(
    {
        "amp": 1,
        "e1": 0.01,
        "e2": 0.01,
        "R_sersic": 0.03,
        "n_sersic": 0.01,
        "center_x": 0.01,
        "center_y": 0.01,
    }
)
kwargs_lower_lens_light.append(
    {
        "amp": 0,
        "e1": -0.5,
        "e2": -0.5,
        "R_sersic": 0.001,
        "n_sersic": 0.5,
        "center_x": -10.0,
        "center_y": -10,
    }
)
kwargs_upper_lens_light.append(
    {
        "amp": 100,
        "e1": -0.05,
        "e2": 0.05,
        "R_sersic": 10.0,
        "n_sersic": 5.0,
        "center_x": 10.0,
        "center_y": 10,
    }
)

lens_light_params = [
    kwargs_lens_light_init,
    kwargs_lens_light_sigma,
    fixed_lens_light,
    kwargs_lower_lens_light,
    kwargs_upper_lens_light,
]

# Point source priors
fixed_ps = [{}]
kwargs_ps_init = []
kwargs_ps_sigma = []
kwargs_lower_ps = []
kwargs_upper_ps = []

init_ra = np.array([0.69920543, -0.69893762, -2.44616562, 2.42290681])
init_dec = np.array([-2.78394548, 2.72912308, -0.41755671, 0.55430711])

kwargs_ps_init.append(
    {
        "ra_image": init_ra,
        "dec_image": init_dec,
        "source_amp": 1,
    }
)
kwargs_ps_sigma.append(
    {
        "ra_image": [0.1, 0.1, 0.1, 0.1],
        "dec_image": [0.1, 0.1, 0.1, 0.1],
        "source_amp": 0.1,
    }
)
kwargs_lower_ps.append(
    {
        "ra_image": init_ra - 0.55,
        "dec_image": init_dec - 0.55,
        "source_amp": 0.5,
    }
)
kwargs_upper_ps.append(
    {
        "ra_image": init_ra + 0.55,
        "dec_image": init_dec + 0.55,
        "source_amp": 10,
    }
)

point_source_params = [
    kwargs_ps_init,
    kwargs_ps_sigma,
    fixed_ps,
    kwargs_lower_ps,
    kwargs_upper_ps,
]

kwargs_params = {
    "lens_model": lens_params,
    "source_model": source_params,
    "lens_light_model": lens_light_params,
    "point_source_model": point_source_params,
}

## FittingSequence
This class holds all of the image generation choices, the data image, the likelihood calculation choices, the choices for which methods are used to sample through the parameter space, and the current distribution/best fit parameters (updated after each sampler finishes running), which are currently set to the prior distributions defined above.

We compare the performances of the jaxtronomy modeling pipeline to that of lenstronomy.

In [ ]:
kwargs_likelihood = {
    "check_bounds": True,  # Checks if sampler goes out of bounds during sampling
    "image_likelihood": True,
    "image_position_likelihood": False,
    "source_position_likelihood": True,
    "astrometric_likelihood": False,
    "source_position_sigma": 0.001,
    "check_positive_flux": True,
    # The following likelihoods are not supported in jaxtronomy yet
    "time_delay_likelihood": False,
    "tracer_likelihood": False,
    "flux_ratio_likelihood": False,
    "kinematic_2d_likelihood": False,
}
kwargs_model = {
    "lens_model_list": lens_model_list,
    "source_light_model_list": source_model_list,
    "lens_light_model_list": lens_light_model_list,
    "point_source_model_list": point_source_type_list,
    "fixed_magnification_list": [True],
}

multi_band_list = [[kwargs_data, kwargs_psf_gaussian, kwargs_numerics]]
kwargs_data_joint = {
    "multi_band_list": multi_band_list,
    # Only single-band is supported in jaxtronomy
    "multi_band_type": "single-band",
}

linear_solver = True
kwargs_constraints = {
    "linear_solver": linear_solver,
    "num_point_source_list": [4],  # Our LENSED_POSITION point source model has 4 images
}

fitting_seq = FittingSequence(
    kwargs_data_joint,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
)
fitting_seq_ref = FittingSequence_ref(
    kwargs_data_joint,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
)

# Store these for later comparison
kwargs_initial = copy.deepcopy(fitting_seq._updateManager.parameter_state)


fitting_kwargs_list_mcmc = [
    ["PSO", {"sigma_scale": 1.0, "n_particles": 50, "n_iterations": 100}],
    # ["MCMC", {"n_burn": 50, "n_run": 50, "n_walkers": 70, "sigma_scale": 1.0}],
]

In [ ]:
# Run the lenstronomy fitting sequence for PSO + MCMC
chain_list = fitting_seq_ref.fit_sequence(fitting_kwargs_list_mcmc)

In [ ]:
# Run the jaxtronomy fitting sequence for PSO + MCMC
chain_list = fitting_seq.fit_sequence(fitting_kwargs_list_mcmc)
kwargs_result = fitting_seq.best_fit()
kwargs_result.pop("kwargs_tracer_source", None)

All of the speedup we got was simply from JAX compiling the functions and optimizing the way the code is run. In this case, we see a 2x speedup using the EPL profile on CPU, which was originally slow due to having to evaluate hyp2f1 for deflection angle calculations. For a simple SIE profile which already runs quickly, the performance may have negligible improvement or become worse, depending on compile time. For GPU, the speedup varies.

## Minimizer
JAX's autodifferentiation feature allows us to make use of a minimizer. Here we use Scipy's BFGS minimization method with 5 chains ran sequentially. The initial parameters for each chain are randomly drawn from the prior distribution, which helps avoid getting stuck at local minima. For posterior sampling, the PSO step can be replaced with this step for a significant speedup.

Support for parallelization is planned.

In [ ]:
# Reset the initial parameters
fitting_seq = FittingSequence(
    kwargs_data_joint,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
)

# Create a copy of the initial set of parameters for later comparison
kwargs_initial = copy.deepcopy(fitting_seq._updateManager.parameter_state)

# options are BFGS and TNC
# Other options such as Nelder-Mead, Powell, CG, Newton-CG, L-BFGS-B, COBYLA, SLSQP, trust-constr, dogleg, trust-ncg, trust-exact, trust-krylov have not been tested
fitting_kwargs_list_jaxopt = [
    [
        "Jaxopt",
        {
            "method": "BFGS",
            "maxiter": 500,
            "rng_int": 420,
            "num_chains": 5,
            "tolerance": 5500,  # We can stop running chains once we get a "good enough" fit
        },
    ],
]

chain_list = fitting_seq.fit_sequence(fitting_kwargs_list_jaxopt)

kwargs_result = fitting_seq.best_fit()
kwargs_result.pop("kwargs_tracer_source", None)

## Analyzing minimizer results

In [ ]:
col1_spacing = 13
col2_spacing = 26
col3_spacing = 26
col4_spacing = 26

print(
    "|"
    + "parameter".center(col1_spacing)
    + "|"
    + "kwargs_initial".center(col2_spacing)
    + "|"
    + "kwargs_result".center(col3_spacing)
    + "|"
    + "kwargs_truth".center(col4_spacing)
    + "|"
)

# Iterate through lens/source/ps
for key in kwargs_truth.keys():
    print(
        "|"
        + "-".center(col1_spacing, "-")
        + "|"
        + "-".center(col2_spacing, "-")
        + "|"
        + "-".center(col3_spacing, "-")
        + "|"
        + "-".center(col4_spacing, "-")
        + "|"
    )
    for i in range(len(kwargs_truth[key])):
        for parameter in kwargs_truth[key][i]:
            if (
                parameter in ["amp", "source_amp", "point_amp"]
                and linear_solver is True
            ):
                continue
            kwargs_initial[key][i][parameter] = np.round(
                kwargs_initial[key][i][parameter], 2
            )
            kwargs_result[key][i][parameter] = np.round(
                kwargs_result[key][i][parameter], 2
            )
            kwargs_truth[key][i][parameter] = np.round(
                kwargs_truth[key][i][parameter], 2
            )

            print(
                "|"
                + parameter.center(col1_spacing)
                + "|"
                + str(kwargs_initial[key][i][parameter]).center(col2_spacing)
                + "|"
                + str(kwargs_result[key][i][parameter]).center(col3_spacing)
                + "|"
                + str(kwargs_truth[key][i][parameter]).center(col4_spacing)
                + "|"
            )

In [ ]:
if linear_solver:
    fitted_image, _, _, amps = (
        fitting_seq.likelihoodModule.image_likelihood.imSim.image_linear_solve(
            **kwargs_result
        )
    )
    print("linearly solved amplitudes:", amps)
else:
    fitted_image = imageModel.image(**kwargs_result)

kwargs_initial.pop("kwargs_tracer_source", None)
initial_image = imageModel.image(**kwargs_initial)

reduced_residual = (fitted_image - image_model) / np.sqrt(
    imageModel.Data.C_D_model(fitted_image)
)

images = [initial_image, fitted_image, image_model, reduced_residual]
titles = ["initial image", "fitted image", "truth image", "reduced residual"]

fig, ax = plt.subplots(1, 4)
for i in range(4):
    ax[i].set_title(titles[i])
    if i in [1, 2]:
        im = ax[i].imshow(images[i], vmin=0, vmax=4)
    else:
        im = ax[i].imshow(images[i])
    plt.colorbar(im, ax=ax[i])

fig.set_figwidth(25)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3))
ax.plot(range(len(chain_list[-1][2])), -np.array(chain_list[-1][2]))
ax.set_ylabel("negative logL", fontsize=14)
ax.set_xlabel("Iteration", fontsize=14)
plt.show()